In [ ]:
import os
os.environ["KAGGLE_CONFIG_DIR"]="/content/kaggle.json"

In [ ]:
import cv2
import numpy as np

In [ ]:
!kaggle datasets download -d sovitrath/diabetic-retinopathy-224x224-gaussian-filtered

Dataset URL: https://www.kaggle.com/datasets/sovitrath/diabetic-retinopathy-224x224-gaussian-filtered
License(s): CC0-1.0
diabetic-retinopathy-224x224-gaussian-filtered.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip diabetic-retinopathy-224x224-gaussian-filtered.zip

Archive:  diabetic-retinopathy-224x224-gaussian-filtered.zip
replace gaussian_filtered_images/gaussian_filtered_images/Mild/0024cdab0c1e.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
mainpath="/content/gaussian_filtered_images/gaussian_filtered_images"
subs=os.listdir(mainpath)
subs.remove('export.pkl')
x=[]
y=[]
for path in subs:
  subspath=os.path.join(mainpath,path)

  imgpath=os.listdir(subspath)

  for img in imgpath:
    imgpath2=os.path.join(subspath,img)
    img=cv2.imread(imgpath2,0)
    # print(img.shape)
    reshaped=img.reshape(224,224,1)
    x.append(reshaped)
    y.append(path)

In [ ]:
print(len(x))
print(len(y))

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.transform(y_test)

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 1)))
model.add(MaxPool2D(2, 2))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPool2D(2, 2))
model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPool2D(2, 2))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(5, activation="softmax"))

In [ ]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [ ]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [ ]:
x_train.shape

In [ ]:
model.fit(x_train,y_train,epochs=10)

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
predpath="/content/gaussian_filtered_images/gaussian_filtered_images/Severe/0c917c372572.png"
img=cv2.imread(predpath,0)
img=img.reshape(1,224,224,1)
model.predict(img)
idx=np.argmax(model.predict(img))
idx=le.inverse_transform([idx])
print(idx)


In [ ]:
!pip install anvil-uplink

In [ ]:
import anvil.server

anvil.server.connect("server_26BEFHUEIAZKRXTW2Y4TM2VN-7UDYCKLMKOYLHVHZ")

In [ ]:
import anvil.media
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder

@anvil.server.callable
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (224, 224))
    # Normalize pixel values
    img = img.astype('float32') / 255

    # Reshape the image to match the expected input shape
    img = img.reshape(1, 224, 224, 1)

    return img

@anvil.server.callable
def classify_image(file):
    # Load your model and label encoder here
    # Example:
    # model = load_model('path_to_your_model')
    # le = LabelEncoder()
    # le.classes_ = np.load('path_to_your_label_encoder_classes.npy')

    # Save the uploaded image to a temporary file
    with anvil.media.TempFile(file) as temp_file:
        # Preprocess the image
        img = preprocess_image(temp_file)

        # Make prediction using the model
        pred = model.predict(img)

        # Get the index of the predicted class
        idx = np.argmax(pred)

        # Convert the index to the class label using the LabelEncoder
        class_label = le.inverse_transform([idx])[0]

        return class_label
